In [1]:
# 수정 후

import os

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용
device = torch.device("cuda:0")

In [2]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
NVIDIA TITAN RTX


In [3]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_v1.zip
using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
import pandas as pd
tag_data = pd.read_csv('./tag_bio_jaso.csv')
tag_data.columns = ['Sentence #', 'word', 'bio_tag']

In [5]:
import math
# data[data.tag==math.nan]
tag_data.bio_tag.unique()

array(['O', 'PER_B', 'PER_I', 'ADDR_B', 'ADDR_I', 'BIR_B', 'BIR_I',
       'MON_B', 'MON_I'], dtype=object)

In [6]:
tag_data.loc[(tag_data['bio_tag'] == "PER"), 'bio_tag'] = 0
tag_data.loc[(tag_data['bio_tag'] == "PER_B"), 'bio_tag'] = 0
tag_data.loc[(tag_data['bio_tag'] == "PER_I"), 'bio_tag'] = 0
tag_data.loc[(tag_data['bio_tag'] == "ADDR"), 'bio_tag'] = 1
tag_data.loc[(tag_data['bio_tag'] == "ADDR_B"), 'bio_tag'] = 1
tag_data.loc[(tag_data['bio_tag'] == "ADDR_I"), 'bio_tag'] = 1
tag_data.loc[(tag_data['bio_tag'] == "MON"), 'bio_tag'] = 2
tag_data.loc[(tag_data['bio_tag'] == "MON_B"), 'bio_tag'] = 2
tag_data.loc[(tag_data['bio_tag'] == "MON_I"), 'bio_tag'] = 2
tag_data.loc[(tag_data['bio_tag'] == "BIR"), 'bio_tag'] = 3
tag_data.loc[(tag_data['bio_tag'] == "BIR_B"), 'bio_tag'] = 3
tag_data.loc[(tag_data['bio_tag'] == "BIR_I"), 'bio_tag'] = 3
tag_data.loc[(tag_data['bio_tag'] == "O"), 'bio_tag'] = 4


data_list = []
for q, label in zip(tag_data['word'], tag_data['bio_tag']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

# for d in data_list:
#     if d[1] in 'nan':
#         print(d)

In [7]:
tag_data.bio_tag.unique()

array([4, 0, 1, 3, 2], dtype=object)

In [8]:
# # train, test 데이터로 나누기 Word
# from sklearn.model_selection import train_test_split
# import random

# # random.Random(len(data_list)).shuffle(data_list)
                                                         
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.200006, random_state=42)

In [9]:
# # train, test 데이터로 나누기 char
# from sklearn.model_selection import train_test_split
# import random

# # random.Random(len(data_list)).shuffle(data_list)
                                                         
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.2000085, random_state=42)

In [10]:
# train, test 데이터로 나누기 jaso
from sklearn.model_selection import train_test_split
import random

# random.Random(len(data_list)).shuffle(data_list)
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.200005, random_state=42)

In [11]:
dataset_test = dataset_test[:-27]

print(len(dataset_train))
print(len(dataset_test))

7547264
1886848


In [12]:
per = 0
addr = 0
mon = 0
bir = 0
o = 0

for d in dataset_train:
    if str(d[1]) == str(0):
        per += 1
    if str(d[1]) == str(1):
        addr += 1
    if str(d[1]) == str(2):
        mon += 1
    if str(d[1]) == str(3):
        bir += 1
    if str(d[1]) == str(4):
        o += 1
        
print('per', per)
print('addr', addr)
print('mon', mon)
print('bir', bir)
print('o', o)

per 2719342
addr 71299
mon 1366984
bir 1554418
o 1835221


In [13]:
per = 0
addr = 0
mon = 0
bir = 0
o = 0

for d in dataset_test:
    if str(d[1]) == str(0):
        per += 1
    if str(d[1]) == str(1):
        addr += 1
    if str(d[1]) == str(2):
        mon += 1
    if str(d[1]) == str(3):
        bir += 1
    if str(d[1]) == str(4):
        o += 1
        
print('per', per)
print('addr', addr)
print('mon', mon)
print('bir', bir)
print('o', o)

per 680821
addr 17686
mon 342131
bir 387933
o 458277


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.word = [transform([str(i[sent_idx])]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.word[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 380
learning_rate =  5e-5

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
data_train[0]

(array([  2, 553,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       dtype=int32),
 array(3, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model = BERTClassifier(bertmodel,  dr_rate=0.5)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

optimizer = BertAdam(optimizer_grouped_parameters, lr=learning_rate, warmup=-1, t_total=-1,
                 b1=0.9, b2=0.999, e=1e-6, weight_decay=0.01,
                 max_grad_norm=1.0)

# class_weights = [9518, 4134, 13873, 12537, 21314]
# normedWeights = [1 - (x / sum(class_weights)) for x in class_weights]
# normedWeights = torch.FloatTensor(normedWeights).to(device)

# loss_fn = nn.CrossEntropyLoss(weight=normedWeights)

loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

t_total value of -1 results in schedule not being applied


In [ ]:
# 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        # token_ids = token_ids.long()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        # token_ids = token_ids.long()
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long()
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        # label = label.long()
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_123676/1621645026.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/117926 [00:00<?, ?it/s]

/etc/anaconda3/envs/test/lib/python3.9/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


epoch 1 batch id 1 loss 1.6085920333862305 train acc 0.234375
epoch 1 batch id 381 loss 1.3370451927185059 train acc 0.33968996062992124
epoch 1 batch id 761 loss 1.0826455354690552 train acc 0.43035479632063073
epoch 1 batch id 1141 loss 0.8616599440574646 train acc 0.4768021472392638
epoch 1 batch id 1521 loss 0.8916258811950684 train acc 0.5037906804733728
epoch 1 batch id 1901 loss 0.756023645401001 train acc 0.52014564702788
epoch 1 batch id 2281 loss 0.907560408115387 train acc 0.5305923936869794
epoch 1 batch id 2661 loss 0.8068732619285583 train acc 0.5383197106350995
epoch 1 batch id 3041 loss 0.8482539653778076 train acc 0.5442442864189412
epoch 1 batch id 3421 loss 0.71912682056427 train acc 0.5491586889798304
epoch 1 batch id 3801 loss 0.8306897878646851 train acc 0.5529137069192318
epoch 1 batch id 4181 loss 0.9292442798614502 train acc 0.5565766264051663
epoch 1 batch id 4561 loss 0.8511300683021545 train acc 0.5595195680771761
epoch 1 batch id 4941 loss 0.802917540073394

epoch 1 batch id 41421 loss 0.9719656109809875 train acc 0.5901822143357234
epoch 1 batch id 41801 loss 0.8264890909194946 train acc 0.590190351307385
epoch 1 batch id 42181 loss 0.695038914680481 train acc 0.5902361252696712
epoch 1 batch id 42561 loss 0.895255446434021 train acc 0.5902601560113719
epoch 1 batch id 42941 loss 0.7835972309112549 train acc 0.590219356209683
epoch 1 batch id 43321 loss 0.9213414192199707 train acc 0.5901900925648069
epoch 1 batch id 43701 loss 1.081103801727295 train acc 0.590078030251024
epoch 1 batch id 44081 loss 0.992877185344696 train acc 0.5894039523831129
epoch 1 batch id 44461 loss 1.0277416706085205 train acc 0.5887603742605879
epoch 1 batch id 44841 loss 1.2978284358978271 train acc 0.5875001393813697
epoch 1 batch id 45221 loss 1.253525733947754 train acc 0.5863170595519781
epoch 1 batch id 45601 loss 1.0783226490020752 train acc 0.5856796588890595
epoch 1 batch id 45981 loss 1.0833799839019775 train acc 0.5850419194884844
epoch 1 batch id 463

...

epoch 2 test acc 0.20559843718200937


  0%|          | 0/117926 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 1.1323870420455933 train acc 0.5
epoch 3 batch id 381 loss 1.0522063970565796 train acc 0.5181266404199475
epoch 3 batch id 761 loss 1.075239658355713 train acc 0.5202858081471747
epoch 3 batch id 1141 loss 1.1320888996124268 train acc 0.5189526730937774
epoch 3 batch id 1521 loss 1.1419003009796143 train acc 0.5199087771203156
epoch 3 batch id 1901 loss 0.9633008241653442 train acc 0.5197182403997895
epoch 3 batch id 2281 loss 1.0499897003173828 train acc 0.5199062911003945
epoch 3 batch id 2661 loss 1.1834346055984497 train acc 0.519858605787298
epoch 3 batch id 3041 loss 0.9525055885314941 train acc 0.5203520634659652
epoch 3 batch id 3421 loss 1.0591604709625244 train acc 0.5202015127155802
epoch 3 batch id 3801 loss 1.2669360637664795 train acc 0.519719317284925
epoch 3 batch id 4181 loss 1.2200915813446045 train acc 0.5197209100693614
epoch 3 batch id 4561 loss 0.9123470783233643 train acc 0.5196434444200834
epoch 3 batch id 4941 loss 1.0299144983291626 tr

epoch 3 batch id 41421 loss 1.1318968534469604 train acc 0.5222437742932329
epoch 3 batch id 41801 loss 0.933627724647522 train acc 0.5222710730604531
epoch 3 batch id 42181 loss 0.8616572618484497 train acc 0.5222708387662691
epoch 3 batch id 42561 loss 1.114953637123108 train acc 0.522300712506755
epoch 3 batch id 42941 loss 1.0453059673309326 train acc 0.5222922148995133
epoch 3 batch id 43321 loss 1.1099225282669067 train acc 0.5223170488908382
epoch 3 batch id 43701 loss 1.0604658126831055 train acc 0.5223171380517608
epoch 3 batch id 44081 loss 0.9828829169273376 train acc 0.5223462914861278
epoch 3 batch id 44461 loss 0.9485629200935364 train acc 0.5223953296147185
epoch 3 batch id 44841 loss 1.140298843383789 train acc 0.5224728066947659
epoch 3 batch id 45221 loss 1.2266210317611694 train acc 0.5224950797195993
epoch 3 batch id 45601 loss 0.9908553957939148 train acc 0.5225104712615951
epoch 3 batch id 45981 loss 1.1445579528808594 train acc 0.5225252685891999
epoch 3 batch id

epoch 3 batch id 82841 loss 0.9359716773033142 train acc 0.5243538827392233
epoch 3 batch id 83221 loss 1.015337586402893 train acc 0.5243682859494598
epoch 3 batch id 83601 loss 1.0107145309448242 train acc 0.5243969494982117
epoch 3 batch id 83981 loss 0.9468413591384888 train acc 0.5244100972243722
epoch 3 batch id 84361 loss 0.946188747882843 train acc 0.5244210891288629
epoch 3 batch id 84741 loss 1.0942317247390747 train acc 0.524413912686893
epoch 3 batch id 85121 loss 1.0090632438659668 train acc 0.5244236880440785
epoch 3 batch id 85501 loss 1.0491913557052612 train acc 0.5244158328557561
epoch 3 batch id 85881 loss 1.066368818283081 train acc 0.5244493470616318
epoch 3 batch id 86261 loss 0.9466456174850464 train acc 0.5244488746362783
epoch 3 batch id 86641 loss 1.0449713468551636 train acc 0.524472211481862
epoch 3 batch id 87021 loss 0.8774000406265259 train acc 0.5244827757093116
epoch 3 batch id 87401 loss 0.9691448211669922 train acc 0.5245027230809716
epoch 3 batch id 

In [ ]:
torch.save(model.state_dict(), './kobert_jaso_model.pt')

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            tag_data.loc[(tag_data['bio_tag'] == "PER"), 'bio_tag'] = 0
            tag_data.loc[(tag_data['bio_tag'] == "PER_B"), 'bio_tag'] = 0
            tag_data.loc[(tag_data['bio_tag'] == "PER_I"), 'bio_tag'] = 0
            tag_data.loc[(tag_data['bio_tag'] == "ADDR"), 'bio_tag'] = 1
            tag_data.loc[(tag_data['bio_tag'] == "ADDR_B"), 'bio_tag'] = 1
            tag_data.loc[(tag_data['bio_tag'] == "ADDR_I"), 'bio_tag'] = 1
            tag_data.loc[(tag_data['bio_tag'] == "MON"), 'bio_tag'] = 2
            tag_data.loc[(tag_data['bio_tag'] == "MON_B"), 'bio_tag'] = 2
            tag_data.loc[(tag_data['bio_tag'] == "MON_I"), 'bio_tag'] = 2
            tag_data.loc[(tag_data['bio_tag'] == "BIR"), 'bio_tag'] = 3
            tag_data.loc[(tag_data['bio_tag'] == "BIR_B"), 'bio_tag'] = 3
            tag_data.loc[(tag_data['bio_tag'] == "BIR_I"), 'bio_tag'] = 3
            tag_data.loc[(tag_data['bio_tag'] == "O"), 'bio_tag'] = 4

            if np.argmax(logits) == 0:
                test_eval.append("PER")
            elif np.argmax(logits) == 1:
                test_eval.append("ADDR")
            elif np.argmax(logits) == 2:
                test_eval.append("MON")
            elif np.argmax(logits) == 3:
                test_eval.append("BIR")
            elif np.argmax(logits) == 4:
                test_eval.append("O")

        print(">> 태그 : " + test_eval[0])

In [ ]:
end=1
while end ==1 :
  sentence = input("단어 입력 : ")
  if sentence == 0:
    break
  predict(sentence)
  print("\n")